# Лабораторная работа №5
## Выполнил студент группы ФИО ГРУППА

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [61]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

In [39]:
import time

def timer(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        value = func(*args, **kwargs)
        print(f"--- {time.time() - start_time:.6f} ms ---")
        return value
    return wrapper

def userInput(prompt, default):
    u = input(prompt)
    return str(u) if u.strip() else default

def driver (func):
    def wrapper():
        main_string = userInput("Enter the main string: ","Around the world around The World")
        substring = userInput("Enter the substring: ","The World")
        option = userInput("Case sensitive? (Y/N): ","n").lower()
        case_sensitive = True if option == 'y' else False
        print("Text:", main_string, "\nPattern:", substring, "\nCase sensitive:", "yes" if case_sensitive else "no")
        print("Matches found at:", func(main_string, substring, case_sensitive))
    return wrapper

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

In [ ]:
'''Standard python search function'''
@driver
@timer
def find_substring(main_string, substring, case_sensitive=True):
    start = 0
    end = len(main_string)
    search_space = main_string[start:end]
    if not case_sensitive:
        search_space = search_space.lower()
        substring = substring.lower()
    return substring in search_space
find_substring()
'''(returns a bool vlaue)'''

Text: Around the world around The World 
Pattern: The World 
Case sensitive: no
--- 0.000003 ms ---
Matches found at: True


#### Алгоритм Кнута-Морриса-Пратта

In [ ]:
'''
Pattern and text are compared iteratively (each one's pointer increments with every character match)
On a mismatch, the pattern pointer resets to the previously matched character's index
FOR NON-REPETITIVE PATTERNS:
lps[i] is the index in the pattern to which we jump when a mismatch occurs
Example: "world" has no repeating character sequences so we jump to index 0 every time
FOR REPETITIVE PATTERNS:
For each pattern index, an overlap length is calculated that acts as an index
to jump to after a mismatch or a full match

For each (0...i) substring in the pattern, the prefix function looks for 
the longest proper prefix that is also a suffix
(the length is 0 if the prefix and suffix for a given string do not match)
the length of that LPS matches the index in the pattern to which we can safely jump
upon a mismatch or full match without the need to traverse the characters before that index
(LPS length = "hom many characters will be automatically matching before this one")
Pattern ACACBAC lps:[0,0,1,2,0,1,2]
|0:""/""|1:""/""|2:"A"/"A"|3:"AC"/"AC"|4:""/""|5:"A"/"A"|6:"AC"/"AC"|
'''
def construct_lps(pattern, case_sensitive):
    lps = [0] * len(pattern) # pattern-length list of zeroes
    overlap_length = 0 # length of a prefix-suffix
    i = 1
    while i < len(pattern): # iterating though the pattern
        prefix = pattern[i] if case_sensitive else pattern[i].lower() # prefix of patten[i]
        suffix = pattern[overlap_length] if case_sensitive else pattern[overlap_length].lower() # suffix of pattern[i]
        if prefix == suffix:
            overlap_length += 1
            lps[i] = overlap_length
            i += 1
        else:
            if overlap_length != 0:
                overlap_length = lps[overlap_length - 1]
            else:
                lps[i] = 0
                i += 1
    return lps
@driver
@timer
def kmp_search(text, pattern, case_sensitive):
    if not pattern:
        return []  # Edge case: empty pattern    
    lps = construct_lps(pattern, case_sensitive)
    i = 0
    j = 0
    matches = []
    while i < len(text):

        text_char = text[i] if case_sensitive else text[i].lower()
        pattern_char = pattern[j] if case_sensitive else pattern[j].lower()
        if text_char == pattern_char: # text and pattern characters match
            i += 1 # both pointers are incremented
            j += 1
            if j == len(pattern): # entire pattern traversed - full match found
                matches.append(i - j) # current position in text - pattern length = start of matched sequence
                j = lps[j - 1]  # pattern iterator reset to the overlap index for that index of the pattern
        else: # text and pattern characters don't match
            if j != 0:
                j = lps[j - 1] # iterator reset to the last matched sequence for that index
            else:
                i += 1
    return matches

if __name__ == "__main__":
    kmp_search()

    # text = "Around the world around The World"
    # pattern = "The World"
    # print("Text:", text)
    # print("Pattern:", pattern)
    # # Case-sensitive search (default)
    # result_case_sensitive = kmp_search(text, pattern, case_sensitive=True)
    # print("Case-sensitive matches:", result_case_sensitive) 
    # # Case-insensitive search
    # result_case_insensitive = kmp_search(text, pattern, case_sensitive=False)
    # print("Case-insensitive matches:", result_case_insensitive)

Text: Around the world around The World 
Pattern: The World 
Case sensitive: no
--- 0.000037 ms ---
Matches found at: [7, 24]


#### Упрощенный алгоритм Бойера-Мура

In [ ]:
'''
The pattern and the text are aligned and iteratively compared backwards.
When a mismatch occurs, the last occurence in the pattern of the mismatched chracter in the text is looked up.
If the mismatched character is present in the pattern, the pattern is shifted to align that character with the character in the text.
If the mismatched character is not present in the pattern, the pattern is shifted past the mismatched character.
The pattern is iteratively compared with the text again until a perfect match is found.
'''
def badCharHeuristic(string, size):
    badChar = defaultdict(lambda: -1) 
    for i in range(size):
        badChar[ord(string[i])] = i 
        # the indices are ASCII character codes
        # the values are presence of the characters in the pattern
        # i>=0 - rightmost occurence of the character | -1 - character not present
    return badChar
@driver
@timer
def BMsearch(txt, pat, case_sensitive):
    m = len(pat)
    n = len(txt)
    badChar = badCharHeuristic(pat, m)
    s = 0
    matches = []
    while(s <= n-m): # as long as shift size is smaller than len(text)-len(pattern)
        j = m-1 # rightmost char index value is less than pattern length by 1
        while j >= 0:
            pat_char = pat[j] if case_sensitive else pat[j].lower()
            txt_char = txt[s+j] if case_sensitive else txt[s+j].lower()
            if pat_char != txt_char: # break loop when mismatch
                break
            j -= 1 # decrement pattern iterator
        if j < 0: # j = -1 - pattern traversed, full match found
            matches.append(s)
            # shift = pattern length - rightmost occurence of the next character
            s += (m-badChar[ord(txt[s+m] if case_sensitive else txt[s+m].lower())] if s+m < n else 1)
            # Shift the pattern so that the next character in the text
            # aligns with its rightmost occurence in the pattern
            # The s+m<n condition prevents going out of range
            # The m-badChar is for shifting past the character
            # not present in the pattern (shift by len(pattern)+1)
        else: # pattern iteration incomplete, mismatch occured
            s += max(1, j-badChar[ord(txt[s+j] if case_sensitive else txt[s+j].lower())])
            # Shift the pattern so that the mismatched character in the text
            # aligns with its rightmost occurence in the pattern
            # The max function prevents shifting backwards
    return matches
    
if __name__ == "__main__":
    BMsearch()

    # text = "Around the world around The World"
    # pattern = "The World"
    # print("Text:", text, "\nPattern:", pattern)
    # print("Matches found at:", BMsearch(text, pattern, False))

Text: Around the world around The World 
Pattern: The World
--- 0.000025 ms ---
Matches found at: [7, 24]


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
##### Входные данные:
Массив чисел, представляющий собой расстановку в порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
##### Выходные данные: 
Если решения нет, то функция должна вернуть пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

### Вывод